# Tuning Fufi hyperparameters with W&B Sweep 🧹 🐶
The idea is automatically tuning hyperparameters with the sweep by the means of a `grid `search.
In order not to spend an eternity doing this thing, we start with a` random` search, and then put a `grid `search on it.

## W&B Setup
🪄 Install `wandb` library and login

Start by installing the library and logging in to your free account.

In [1]:
!pip install wandb -qU
# Log in to your W&B account
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Installing libraries 📚

In [1]:
!pip install gym==0.25.2
#needed from March
!pip install numpy==1.23.5

## Setting things up for the environment 🌍 🪖

In [2]:
import os

#saving current directory just to be sure
content_dir = os.getcwd()

#cloning Fufi repo from git
!git clone https://github.com/Gaianeve/gym-Fufi.git
#installing things
!pip install /content/gym-Fufi

Cloning into 'gym-Fufi'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 194 (delta 36), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (194/194), 70.91 KiB | 756.00 KiB/s, done.
Resolving deltas: 100% (59/59), done.
Processing ./gym-Fufi
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
# Enter the environment directory
%cd /content/gym-Fufi
# Actually importing the library for our environment
import gym_Fufi

/content/gym-Fufi


In [4]:
#get back to content directory so I save everything there
%cd ..
!pwd

/content
/content


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## importing libraries and functions 📚


In [5]:
#libraries
import argparse
import random
import time
from distutils.util import strtobool
import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## Log needed function from files 📡
Loading files directly from git, so I don't have to upload it by hand.

In [6]:
#get files from git
!git clone https://github.com/Gaianeve/FUFONE.git

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'FUFONE'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 313 (delta 98), reused 4 (delta 4), pack-reused 151
Receiving objects: 100% (313/313), 8.39 MiB | 7.71 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [7]:
!pwd
%cd FUFONE/PPO
from environment import vectorize_env
from agent_class import Agent
from agent_utils import anneal, collect_data, GAE, PPO_train_agent, evaluate_agent

#back to main directory
%cd ..
#import main function from file
from main_function_sweep import main, parse_args

/content
/content/FUFONE/PPO
/content/FUFONE


In [8]:
#back to content directory
%cd ..

/content


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Define the sweep 📑
I assume to start with a random search, that is why we take random distribution:
*   `ent_coef ` ➡ Discrete uniform distribution on integers. Between 0.01, that is the default value and 1, that is the thing that works best for now.
*  ` num_envs`  ➡ list of values to try out
*   `learning_rate` ➡  Quantized log uniform. Returns `round(X / q) * q` where `X` is` log_uniform_values`. Basically, a pretty good approximation of a log_q uniform distribution


In [9]:
import numpy as np
import random

# Define sweep config
sweep_configuration = {
    "method": "random",
    "name": "sweep_Fufi",
    "metric": {"goal": "maximize", "name": "sum_episodes"},
    "parameters": {
       "num_steps": {"values": [128, 256, 512, 2048, 4096, 8192]},
       "ent_coef": {'distribution': 'uniform', 'min': 0.01,'max': 1},
       "num_envs" : {"values": [2,4,8,16,32]},
       "learning_rate":  {'distribution': 'uniform', 'min': 1.5e-7,'max': 1.5e-4}
    },
}


In [10]:
#print the result
import pprint
pprint.pprint(sweep_configuration)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'sum_episodes'},
 'name': 'sweep_Fufi',
 'parameters': {'ent_coef': {'distribution': 'uniform', 'max': 1, 'min': 0.01},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.00015,
                                  'min': 1.5e-07},
                'num_envs': {'values': [2, 4, 8, 16, 32]},
                'num_steps': {'values': [128, 256, 512, 2048, 4096, 8192]}}}


## Run main with the sweep 🏃 🧹
The `wandb.sweep` function initializes the sweep using the configuration. The `wandb.agent `function runs the sweep, executing the `sweep_main` function for each set of parameters.

📚 **Handling Parameters in Script**: In `sweep_main`, `wandb.init()` initializes a run. The script updates the args with the parameters from the sweep `(wandb.config)`, which are then passed to the main function.

📚 **Note**: Added `if __name__ == "__main__":` This ensures that main is called only when the script is executed directly, not when imported as a module.

In [11]:
import wandb
if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep=sweep_configuration, project="Fufi_sweep_trials")  # Set up the sweep

    def sweep_main():
       with wandb.init() as run:
        # retrieve the parser
        args = parse_args()

        # Update args with sweep parameters
        args.learning_rate = wandb.config.learning_rate
        args.num_steps = wandb.config.num_steps
        args.ent_coef = wandb.config.ent_coef
        args.num_envs = wandb.config.num_envs

        main(args)


/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


Create sweep with ID: ryynsgp2
Sweep URL: https://wandb.ai/cartpole_maria_gaia/Fufi_sweep_trials/sweeps/ryynsgp2


In [12]:
 wandb.agent(sweep_id, function=sweep_main, count = 10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
wandb: Agent Starting Run: xyjiawxp with config:
wandb: 	ent_coef: 0.6686750578772821
wandb: 	learning_rate: 9.51297008074646e-05
wandb: 	num_envs: 32
wandb: 	num_steps: 128
wandb: Currently logged in as: gaiabartoli7 (cartpole_maria_gaia). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/vector/vector_env.py:56: DeprecationWarning: WARN: Initializing vector env in old step API which returns one bool array instead 

dissennatore
12.0
dissennatore
24.0
dissennatore
37.0
dissennatore
52.0
dissennatore
68.0
dissennatore
85.0
dissennatore
102.0
dissennatore
120.0
dissennatore
139.0
dissennatore
159.0
dissennatore
179.0
dissennatore
200.0
dissennatore
221.0
dissennatore
243.0
dissennatore
265.0
dissennatore
291.0
dissennatore
318.0
dissennatore
346.0
dissennatore
375.0
dissennatore
405.0
dissennatore
435.0
dissennatore
448.0
dissennatore
479.0
dissennatore
511.0
dissennatore
544.0
dissennatore
580.0
dissennatore
600.0
dissennatore
619.0
dissennatore
644.0
dissennatore
663.0
dissennatore
702.0
dissennatore
743.0
dissennatore
758.0
dissennatore
800.0
dissennatore
830.0
dissennatore
860.0
dissennatore
886.0
dissennatore
929.0
dissennatore
952.0
dissennatore
974.0
dissennatore
996.0
dissennatore
1041.0
dissennatore
1058.0
dissennatore
1106.0
dissennatore
1124.0
dissennatore
1153.0
dissennatore
1203.0
dissennatore
1218.0
dissennatore
1238.0
dissennatore
1263.0
dissennatore
1290.0
dissennatore
1302.0
dissenn

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will ren

...

dissennatore
296674.0
dissennatore
296747.0
dissennatore
296777.0
dissennatore
296847.0
dissennatore
296940.0
dissennatore
296984.0
dissennatore
297111.0
dissennatore
297132.0
dissennatore
297360.0
dissennatore
297413.0
dissennatore
297437.0
dissennatore
297480.0
dissennatore
297513.0
dissennatore
297637.0
dissennatore
297659.0
dissennatore
297789.0
dissennatore
297944.0
dissennatore
298059.0
dissennatore
298112.0
dissennatore
298179.0
dissennatore
298331.0
dissennatore
298439.0
dissennatore
298487.0
dissennatore
298551.0
dissennatore
298595.0
dissennatore
298849.0
dissennatore
298862.0
dissennatore
299059.0
dissennatore
299116.0
dissennatore
299309.0
dissennatore
299342.0
dissennatore
299414.0
dissennatore
299440.0
dissennatore
299559.0
dissennatore
299584.0
dissennatore
299763.0
dissennatore
299793.0
dissennatore
299837.0
dissennatore
299862.0
dissennatore
299993.0
dissennatore
300033.0
dissennatore
300065.0
dissennatore
300121.0
dissennatore
300182.0
dissennatore
300210.0
dissennato

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


dissennatore
317876.0
dissennatore
317916.0
dissennatore
317994.0
dissennatore
318039.0
dissennatore
318098.0
dissennatore
318128.0
dissennatore
318200.0
dissennatore
318289.0
dissennatore
318355.0
dissennatore
318383.0
dissennatore
318567.0
dissennatore
318663.0
dissennatore
318874.0
dissennatore
318934.0
dissennatore
319000.0
dissennatore
319117.0
dissennatore
319162.0
dissennatore
319194.0
dissennatore
319293.0
dissennatore
319394.0
dissennatore
319412.0
dissennatore
319578.0
dissennatore
319624.0
dissennatore
319662.0
dissennatore
319769.0
dissennatore
319815.0
dissennatore
319845.0
dissennatore
320001.0
dissennatore
320029.0
dissennatore
320051.0
dissennatore
320137.0
dissennatore
320200.0
dissennatore
320233.0
dissennatore
320309.0
dissennatore
320411.0
dissennatore
320424.0
dissennatore
320467.0
dissennatore
320482.0
dissennatore
320561.0
dissennatore
320603.0
dissennatore
320628.0
dissennatore
320668.0
dissennatore
320759.0
dissennatore
320794.0
dissennatore
320859.0
dissennato

wandb: Ctrl + C detected. Stopping sweep.



325392.0
dissennatore

In [14]:
import shutil
shutil.rmtree('FUFONE')